# Imports

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [129]:
import datetime
import pandas as pd
import seaborn as sns
import numpy as np
import datetime as dt

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import BatchNormalization, Dense, Input, Dropout
from tensorflow_addons.metrics import RSquare
from keras.models import Model
from keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LassoCV

In [3]:
pd.set_option('display.max_columns', 500)

# Data Load

In [5]:
!pwd

/Users/lj/code/5pacepenguin/nba_betting_analysis/nba_betting_analysis/backend/notebooks


In [92]:
df = pd.read_pickle('/Users/lj/code/5pacepenguin/nba_betting_analysis/nba_betting_analysis/backend/data/pkl/scaled_boxscore_advanced_rolling_players.pkl')
df

,PLAYER_ID,GAME_DATE,PLAYER_NAME,PLUS_MINUS,AST_PCT,AST_RATIO,AST_TOV,DEF_RATING,DREB_PCT,EFG_PCT,MIN,NET_RATING,OFF_RATING,OREB_PCT,PACE,PIE,POSS,PTS,REB_PCT,TM_TOV_PCT,TS_PCT,USG_PCT
0,1713,2018-12-03,Vince Carter,-14.9,0.123266,0.169653,0.042814,0.578202,0.250375,0.402074,0.329102,0.501794,0.587002,0.050330,0.014105,0.542849,0.347285,0.421875,0.228959,0.072430,0.431366,0.376921
1,1713,2018-12-05,Vince Carter,-14.8,0.132462,0.147789,0.042814,0.546110,0.255056,0.432649,0.366260,0.513995,0.573205,0.064214,0.013059,0.548627,0.383484,0.437500,0.231373,0.072430,0.465736,0.384718
2,1713,2018-12-08,Vince Carter,-13.9,0.147144,0.161434,0.057086,0.548544,0.268539,0.466538,0.354895,0.515012,0.577199,0.083652,0.013363,0.554863,0.373303,0.437500,0.249472,0.090059,0.499260,0.418802
3,1713,2018-12-12,Vince Carter,-13.9,0.125686,0.163571,0.014271,0.554043,0.256742,0.414582,0.347681,0.500778,0.561224,0.065255,0.014036,0.548169,0.368778,0.445312,0.231071,0.070731,0.447864,0.404767
4,1713,2018-12-14,Vince Carter,-11.5,0.125686,0.163571,0.014271,0.595691,0.256742,0.396729,0.346915,0.476435,0.566125,0.047900,0.013775,0.547654,0.367647,0.474609,0.221418,0.070731,0.422377,0.418133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100079,1631323,2023-02-23,Simone Fontecchio,0.6,0.092611,0.167352,0.071357,0.566844,0.141199,0.415758,0.251965,0.588337,0.706635,0.133287,0.012827,0.522883,0.263575,0.759766,0.171342,0.267842,0.421003,0.387391
100080,1631323,2023-02-25,Simone Fontecchio,2.3,0.107293,0.190860,0.071357,0.579735,0.141199,0.429121,0.264539,0.622010,0.770627,0.152725,0.012945,0.538387,0.277149,0.761719,0.182202,0.214741,0.455690,0.367788
100081,1631323,2023-02-28,Simone Fontecchio,-0.8,0.128751,0.218313,0.080919,0.630758,0.094382,0.375668,0.305364,0.582356,0.761732,0.152725,0.012803,0.522654,0.319005,0.699219,0.139065,0.267842,0.402813,0.317665
100082,1631323,2023-03-03,Simone Fontecchio,-5.3,0.139561,0.228670,0.095191,0.612729,0.045880,0.277635,0.358884,0.601974,0.773260,0.096147,0.011583,0.523112,0.369910,0.666016,0.080543,0.139550,0.305626,0.312542


## Modelling column selection and split

In [22]:
game_date_range = df.GAME_DATE.max() - df.GAME_DATE.min()

In [54]:
train_valid_split_date = df.GAME_DATE.min() + datetime.timedelta(np.round(game_date_range.days*0.8))

In [55]:
train_valid_split_date

Timestamp('2022-04-27 00:00:00')

In [51]:
valid_test_split_delta = (df.GAME_DATE.max() - train_test_split_date) / 2

In [60]:
valid_test_split_date = df.GAME_DATE.max() - (valid_test_split_delta / 2)
valid_test_split_date

Timestamp('2022-12-17 00:00:00')

In [93]:
X_train = df.loc[df.GAME_DATE < train_valid_split_date, ~df.columns.isin(['PLUS_MINUS'])].copy()
X_valid = df.loc[(df.GAME_DATE > train_valid_split_date) & (df.GAME_DATE < valid_test_split_date), ~df.columns.isin(['PLUS_MINUS'])].copy()
X_test = df.loc[df.GAME_DATE > valid_test_split_date, ~df.columns.isin(['PLUS_MINUS'])].copy()
y_train = df.loc[df.GAME_DATE < train_valid_split_date, ['PLUS_MINUS']].copy()
y_valid = df.loc[(df.GAME_DATE > train_valid_split_date) & (df.GAME_DATE < valid_test_split_date), ['PLUS_MINUS']]
y_test = df.loc[df.GAME_DATE > valid_test_split_date, ['PLUS_MINUS']].copy()

In [94]:
X_train, X_valid, X_test, y_train, y_valid, y_test

(      PLAYER_ID  GAME_DATE   PLAYER_NAME   AST_PCT  AST_RATIO   AST_TOV  \
 0          1713 2018-12-03  Vince Carter  0.123266   0.169653  0.042814   
 1          1713 2018-12-05  Vince Carter  0.132462   0.147789  0.042814   
 2          1713 2018-12-08  Vince Carter  0.147144   0.161434  0.057086   
 3          1713 2018-12-12  Vince Carter  0.125686   0.163571  0.014271   
 4          1713 2018-12-14  Vince Carter  0.125686   0.163571  0.014271   
 ...         ...        ...           ...       ...        ...       ...   
 98794   1630846 2022-03-28  Olivier Sarr  0.102775   0.245602  0.085629   
 98795   1630846 2022-03-30  Olivier Sarr  0.124234   0.267467  0.099900   
 98796   1630846 2022-04-01  Olivier Sarr  0.121007   0.242643  0.114172   
 98797   1630846 2022-04-03  Olivier Sarr  0.135689   0.253658  0.121307   
 98798   1630846 2022-04-05  Olivier Sarr  0.123104   0.223738  0.092764   
 
        DEF_RATING  DREB_PCT   EFG_PCT       MIN  NET_RATING  OFF_RATING  \
 0        

# Models

## Linear Baseline

In [137]:
lasso_cv = LassoCV(random_state=42, cv=10, n_jobs=-1, max_iter=10000)

In [138]:
lasso_cv.fit(pd.concat((X_train_modelling, X_valid_modelling), axis=0), pd.concat((y_train, y_valid), axis=0))

/opt/homebrew/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:1568: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LassoCV(cv=10, max_iter=10000, n_jobs=-1, random_state=42)

In [139]:
lasso_cv.__dict__

{'eps': 0.001,
 'n_alphas': 100,
 'alphas': None,
 'fit_intercept': True,
 'precompute': 'auto',
 'max_iter': 10000,
 'tol': 0.0001,
 'copy_X': True,
 'cv': 10,
 'verbose': False,
 'n_jobs': -1,
 'positive': False,
 'random_state': 42,
 'selection': 'cyclic',
 'feature_names_in_': array(['AST_PCT', 'AST_RATIO', 'AST_TOV', 'DEF_RATING', 'DREB_PCT',
        'EFG_PCT', 'MIN', 'NET_RATING', 'OFF_RATING', 'OREB_PCT', 'PACE',
        'PIE', 'POSS', 'PTS', 'REB_PCT', 'TM_TOV_PCT', 'TS_PCT', 'USG_PCT'],
       dtype=object),
 'n_features_in_': 18,
 'mse_path_': array([[39.5820389 , 38.93038413, 36.54442871, 38.12964636, 43.51391892,
         43.92924316, 43.54789303, 50.69857506, 56.12177344, 34.09521374],
        [37.83706589, 37.27768592, 35.00889853, 36.58633341, 41.89865336,
         42.31591938, 41.85448166, 49.06955795, 54.81815423, 32.7834459 ],
        [36.28319301, 35.84182329, 33.67707115, 35.24137017, 40.48659288,
         40.61631406, 40.22181929, 47.34893397, 52.79850248, 31.66320

In [140]:
np.sqrt(19.25201705)

4.387712051855727

## Neural Net

In [80]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [125]:
# BASED ON: https://github.com/charlesmalafosse/sports-betting-customloss/blob/master/notebook/BetSentiment_SportsBetting_CustomLossFunction.ipynb

def get_model(input_dim, output_dim, base=1000, multiplier=0.25, p=0.2):
    inputs = Input(shape=(input_dim,))
    l = BatchNormalization()(inputs)
    l = Dropout(p)(l)
    n = base
    l = Dense(n, activation='relu')(l)
    l = BatchNormalization()(l)
    l = Dropout(p)(l)
    n = int(n * multiplier)
    l = Dense(n, activation='relu')(l)
    l = BatchNormalization()(l)
    l = Dropout(p)(l)
    n = int(n * multiplier)
    l = Dense(n, activation='relu')(l)
    outputs = Dense(output_dim, activation='relu')(l)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', 
                  loss=root_mean_squared_error,
                  metrics=[root_mean_squared_error, RSquare()]
                 )
    return model

In [33]:
X_train.loc[:, ~X_train.columns.isin(['PLAYER_ID', 'GAME_DATE', 'PLAYER_NAME'])].shape

(79708, 18)

In [126]:
model = get_model(18, 1, base=2048)
model.layers

In [110]:
es = EarlyStopping(patience=25, restore_best_weights=True)

In [96]:
X_train_modelling = X_train.loc[:, ~X_train.columns.isin(['PLAYER_ID', 'GAME_DATE', 'PLAYER_NAME'])]
X_valid_modelling = X_valid.loc[:, ~X_valid.columns.isin(['PLAYER_ID', 'GAME_DATE', 'PLAYER_NAME'])]
X_test_eval = X_test.loc[:, ~X_test.columns.isin(['PLAYER_ID', 'GAME_DATE', 'PLAYER_NAME'])]

In [127]:
model.fit(X_train_modelling, y_train, 
          batch_size=64,
          callbacks=es,
          epochs=150, 
          validation_data=(X_valid_modelling, y_valid))

Epoch 1/150
1246/1246 [==============================] - 7s 6ms/step - loss: 5.7125 - root_mean_squared_error: 5.7121 - r_square: 0.2444 - val_loss: 4.7974 - val_root_mean_squared_error: 4.7971 - val_r_square: 0.2549
Epoch 2/150
1246/1246 [==============================] - 7s 6ms/step - loss: 5.6443 - root_mean_squared_error: 5.6442 - r_square: 0.2620 - val_loss: 4.7420 - val_root_mean_squared_error: 4.7415 - val_r_square: 0.2668
Epoch 3/150
1246/1246 [==============================] - 7s 6ms/step - loss: 5.6327 - root_mean_squared_error: 5.6321 - r_square: 0.2655 - val_loss: 4.7641 - val_root_mean_squared_error: 4.7637 - val_r_square: 0.2624
Epoch 4/150
1246/1246 [==============================] - 7s 6ms/step - loss: 5.6260 - root_mean_squared_error: 5.6262 - r_square: 0.2674 - val_loss: 4.7617 - val_root_mean_squared_error: 4.7612 - val_r_square: 0.2634
Epoch 5/150
1246/1246 [==============================] - 7s 6ms/step - loss: 5.6130 - root_mean_squared_error: 5.6128 - r_square: 0.

1246/1246 [==============================] - 9s 7ms/step - loss: 5.5304 - root_mean_squared_error: 5.5301 - r_square: 0.2911 - val_loss: 4.7691 - val_root_mean_squared_error: 4.7686 - val_r_square: 0.2629


In [128]:
model.evaluate(X_test_eval, y_test)

296/296 [==============================] - 0s 1ms/step - loss: 4.3363 - root_mean_squared_error: 4.3370 - r_square: 0.2571


[4.336276531219482, 4.336963653564453, 0.2570677399635315]

In [ ]:
sns.scatterplot(model.)

In [102]:
y_test.value_counts()

PLUS_MINUS
 1.5          114
 1.8          108
 1.7          107
 1.2          106
 0.1           91
             ... 
-1.1            1
-1.2            1
-1.3            1
-1.4            1
 16.6           1
Length: 584, dtype: int64

In [91]:
scaler = MinMaxScaler()
y_test_unscaled = pd.concat((y_test, pd.Series(0.06846369057893753)), axis=0)
y_test_unscaled = scaler.inverse_transform(y_test_unscaled)
y_test_unscaled

NotFittedError: This MinMaxScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.